Movie Studio

In the last 10 years, which genre was the most profitable at the box office. profits between genres.

Most popular english movies filtered by user rating??? filters - year, english, gross profit, user rating, best director 3 recommendations? comedy vs drama

In [271]:
from scipy import stats

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import pandas as pd
import numpy as np

import sqlite3
import zipfile

import warnings
warnings.filterwarnings('ignore')

In [272]:
movie_budget = pd.read_csv("data/tn.movie_budgets.csv.gz", sep= ",")

In [273]:
movie_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [274]:
movie_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [275]:
movie_budget_copy = movie_budget.copy()

movie_budget_copy['release_date'] = pd.to_datetime(movie_budget_copy['release_date'], errors='coerce')

movie_budget_copy['release_year'] = pd.to_datetime(movie_budget_copy['release_date']).dt.year

# Extract the month from the 'release_date' column
movie_budget_copy['release_month'] = movie_budget_copy['release_date'].dt.month

movie_budget_copy = movie_budget_copy[movie_budget_copy['release_year'] >= 2000]

# Display the DataFrame with the new 'release_month' column
movie_budget_copy


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009,12
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,5
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,6
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,5
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017,12
...,...,...,...,...,...,...,...,...
5776,77,2004-12-31,The Mongol King,"$7,000",$900,$900,2004,12
5777,78,2018-12-31,Red 11,"$7,000",$0,$0,2018,12
5779,80,2005-07-13,Return to the Land of Wonders,"$5,000","$1,338","$1,338",2005,7
5780,81,2015-09-29,A Plague So Pleasant,"$1,400",$0,$0,2015,9


In [276]:
conn = sqlite3.connect('im.db.zip')
#specify the path to the zip file
zip_file_path = 'data/im.db.zip'

#Extract the SQLite database file from the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('unzippedData')

#connect to extracted SQLite database
database_path = 'unzippedData/im.db'
conn = sqlite3.connect(database_path)

In [277]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", conn)

schema_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [278]:
movie_basic = pd.read_sql('''
    SELECT *
    FROM movie_basics
''', conn)
movie_basic

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [279]:
movie_basic["start_year"].value_counts()

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2027        1
2026        1
2025        1
2115        1
Name: start_year, dtype: int64

In [280]:
result_sql = pd.merge(movie_budget_copy[['release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross', "release_year", "release_month"]], movie_basic[['primary_title', 'original_title', 'start_year', 'genres']], left_on='movie', right_on='primary_title', how='inner')

result_sql

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres
0,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009,12,Avatar,Abatâ,2011,Horror
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,5,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,"Action,Adventure,Fantasy"
2,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,6,Dark Phoenix,Dark Phoenix,2019,"Action,Adventure,Sci-Fi"
3,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,5,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,"Action,Adventure,Sci-Fi"
4,2018-04-27,Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018,4,Avengers: Infinity War,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...
3324,2001-07-06,Cure,"$10,000","$94,596","$94,596",2001,7,Cure,Cure,2011,Drama
3325,2001-07-06,Cure,"$10,000","$94,596","$94,596",2001,7,Cure,Cure,2014,None
3326,2012-01-13,Newlyweds,"$9,000","$4,584","$4,584",2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama"
3327,2018-12-31,Red 11,"$7,000",$0,$0,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller"


In [281]:
result_sql['worldwide_gross'] = result_sql['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(float)
result_sql['production_budget'] = result_sql['production_budget'].str.replace('$', '').str.replace(',', '').astype(float)

result_sql['ROI'] = (result_sql['worldwide_gross'] / result_sql['production_budget']) * 100

result_sql

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
0,2009-12-18,Avatar,425000000.0,"$760,507,625",2.776345e+09,2009,12,Avatar,Abatâ,2011,Horror,653.257713
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,"$241,063,875",1.045664e+09,2011,5,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,"Action,Adventure,Fantasy",254.667286
2,2019-06-07,Dark Phoenix,350000000.0,"$42,762,350",1.497624e+08,2019,6,Dark Phoenix,Dark Phoenix,2019,"Action,Adventure,Sci-Fi",42.789243
3,2015-05-01,Avengers: Age of Ultron,330600000.0,"$459,005,868",1.403014e+09,2015,5,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,"Action,Adventure,Sci-Fi",424.384139
4,2018-04-27,Avengers: Infinity War,300000000.0,"$678,815,482",2.048134e+09,2018,4,Avengers: Infinity War,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi",682.711400
...,...,...,...,...,...,...,...,...,...,...,...,...
3324,2001-07-06,Cure,10000.0,"$94,596",9.459600e+04,2001,7,Cure,Cure,2011,Drama,945.960000
3325,2001-07-06,Cure,10000.0,"$94,596",9.459600e+04,2001,7,Cure,Cure,2014,None,945.960000
3326,2012-01-13,Newlyweds,9000.0,"$4,584",4.584000e+03,2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama",50.933333
3327,2018-12-31,Red 11,7000.0,$0,0.000000e+00,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller",0.000000


In [282]:
year = result_sql[result_sql['start_year'] <= 2020]
year

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
0,2009-12-18,Avatar,425000000.0,"$760,507,625",2.776345e+09,2009,12,Avatar,Abatâ,2011,Horror,653.257713
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000.0,"$241,063,875",1.045664e+09,2011,5,Pirates of the Caribbean: On Stranger Tides,Pirates of the Caribbean: On Stranger Tides,2011,"Action,Adventure,Fantasy",254.667286
2,2019-06-07,Dark Phoenix,350000000.0,"$42,762,350",1.497624e+08,2019,6,Dark Phoenix,Dark Phoenix,2019,"Action,Adventure,Sci-Fi",42.789243
3,2015-05-01,Avengers: Age of Ultron,330600000.0,"$459,005,868",1.403014e+09,2015,5,Avengers: Age of Ultron,Avengers: Age of Ultron,2015,"Action,Adventure,Sci-Fi",424.384139
4,2018-04-27,Avengers: Infinity War,300000000.0,"$678,815,482",2.048134e+09,2018,4,Avengers: Infinity War,Avengers: Infinity War,2018,"Action,Adventure,Sci-Fi",682.711400
...,...,...,...,...,...,...,...,...,...,...,...,...
3324,2001-07-06,Cure,10000.0,"$94,596",9.459600e+04,2001,7,Cure,Cure,2011,Drama,945.960000
3325,2001-07-06,Cure,10000.0,"$94,596",9.459600e+04,2001,7,Cure,Cure,2014,None,945.960000
3326,2012-01-13,Newlyweds,9000.0,"$4,584",4.584000e+03,2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama",50.933333
3327,2018-12-31,Red 11,7000.0,$0,0.000000e+00,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller",0.000000


In [283]:
budget_filter = result_sql[result_sql["production_budget"] <= 80000000.0]
budget_filter

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
211,2009-06-05,Home,12000000.0,$0,0.0,2009,6,Home,Yurt,2011,Drama,0.000000
212,2009-06-05,Home,12000000.0,$0,0.0,2009,6,Home,Home,2011,"Documentary,Drama,Family",0.000000
213,2009-06-05,Home,12000000.0,$0,0.0,2009,6,Home,Dom,2011,Drama,0.000000
214,2009-06-05,Home,12000000.0,$0,0.0,2009,6,Home,Home,2015,"Adventure,Animation,Comedy",0.000000
215,2009-06-05,Home,12000000.0,$0,0.0,2009,6,Home,Hemma,2013,"Drama,Romance",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
3324,2001-07-06,Cure,10000.0,"$94,596",94596.0,2001,7,Cure,Cure,2011,Drama,945.960000
3325,2001-07-06,Cure,10000.0,"$94,596",94596.0,2001,7,Cure,Cure,2014,None,945.960000
3326,2012-01-13,Newlyweds,9000.0,"$4,584",4584.0,2012,1,Newlyweds,Newlyweds,2011,"Comedy,Drama",50.933333
3327,2018-12-31,Red 11,7000.0,$0,0.0,2018,12,Red 11,Red 11,2019,"Horror,Sci-Fi,Thriller",0.000000


In [284]:
profit = budget_filter[budget_filter["ROI"] >= 200.0]
profit

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
235,2009-04-23,Home,500000.0,"$15,433",44793168.0,2009,4,Home,Yurt,2011,Drama,8958.6336
236,2009-04-23,Home,500000.0,"$15,433",44793168.0,2009,4,Home,Home,2011,"Documentary,Drama,Family",8958.6336
237,2009-04-23,Home,500000.0,"$15,433",44793168.0,2009,4,Home,Dom,2011,Drama,8958.6336
238,2009-04-23,Home,500000.0,"$15,433",44793168.0,2009,4,Home,Home,2015,"Adventure,Animation,Comedy",8958.6336
239,2009-04-23,Home,500000.0,"$15,433",44793168.0,2009,4,Home,Hemma,2013,"Drama,Romance",8958.6336
...,...,...,...,...,...,...,...,...,...,...,...,...
3312,2015-09-01,Exeter,25000.0,$0,489792.0,2015,9,Exeter,Exeter,2014,Horror,1959.1680
3313,2015-09-01,Exeter,25000.0,$0,489792.0,2015,9,Exeter,Exeter,2015,None,1959.1680
3323,2006-04-28,Clean,10000.0,"$138,711",138711.0,2006,4,Clean,Clean,2017,"Comedy,Drama,Horror",1387.1100
3324,2001-07-06,Cure,10000.0,"$94,596",94596.0,2001,7,Cure,Cure,2011,Drama,945.9600


In [285]:
profit.drop_duplicates(subset="movie", keep="first", inplace=True)
profit

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,release_month,primary_title,original_title,start_year,genres,ROI
235,2009-04-23,Home,500000.0,"$15,433",44793168.0,2009,4,Home,Yurt,2011,Drama,8958.633600
444,2012-03-23,The Hunger Games,80000000.0,"$408,010,692",677923379.0,2012,3,The Hunger Games,The Hunger Games,2012,"Action,Adventure,Sci-Fi",847.404224
445,2011-05-26,The Hangover Part II,80000000.0,"$254,464,305",586464305.0,2011,5,The Hangover Part II,The Hangover Part II,2011,"Comedy,Mystery",733.080381
446,2017-02-10,The Lego Batman Movie,80000000.0,"$175,750,384",310660018.0,2017,2,The Lego Batman Movie,The Lego Batman Movie,2017,"Action,Animation,Comedy",388.325022
447,2015-09-25,Hotel Transylvania 2,80000000.0,"$169,700,110",469500298.0,2015,9,Hotel Transylvania 2,Hotel Transylvania 2,2015,"Animation,Comedy,Family",586.875373
...,...,...,...,...,...,...,...,...,...,...,...,...
3305,2016-03-18,Krisha,30000.0,"$144,822",144822.0,2016,3,Krisha,Krisha,2015,Drama,482.740000
3310,2000-01-14,The Terrorist,25000.0,"$195,043",195043.0,2000,1,The Terrorist,The Terrorist,2018,Thriller,780.172000
3311,2015-09-01,Exeter,25000.0,$0,489792.0,2015,9,Exeter,Exeter,2015,"Horror,Mystery,Thriller",1959.168000
3323,2006-04-28,Clean,10000.0,"$138,711",138711.0,2006,4,Clean,Clean,2017,"Comedy,Drama,Horror",1387.110000


In [286]:
profit["movie"].value_counts()

The Grey                1
Where the Heart Is      1
Safe Haven              1
Scream 4                1
Yogi Bear               1
                       ..
Closer                  1
The Illusionist         1
The Butterfly Effect    1
Ida                     1
Bridesmaids             1
Name: movie, Length: 731, dtype: int64